In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import sys
import math
import shapely.geometry
from shapely.geometry import Polygon
from shapely.wkt import loads
from shapely.geometry import LineString, Point, shape
pd.set_option('display.max_columns', None)

#### Paths

In [6]:
impact_path = "./../../data/input-data/dish/impacts/"
gis_path = "./../../data/input-data/dish/gis/"
output_path = "./../../data/output-data/dish/denver/final-impacts/"

#### Load impacts_df

In [7]:
impacts_df = pd.read_csv("{}dish-september-2025-impacts.csv".format(impact_path))
impacts_df.head(2)

,tag,cilac,next_cilac,traffic_mobility,traffic_stationary,traffic_unknown,drop_mobility,drop_stationary,drop_unknown,voice_traffic_stationary,voice_traffic_mobility,voice_traffic_unknown,voice_drops_mobility,voice_drops_stationary,voice_drops_unknown,impact_time,drop_impact_time
0,-1,41396777666,41396777815,1,0,0,0,0,0,0,0,0,0,0,0,1,0
1,-1,41437720667,41437720920,35,28,4,0,0,0,1,0,1,0,0,0,567,0


#### load Cell GIS

In [8]:
cells_df = pd.read_csv("{}gis.csv".format(gis_path), names = ['cell_name','cell_cilac', 'SectorID', 'RNC_BSC', 'LAC', 'SectorType', 'Scr_Freq', \
                                                            'UARFCN', 'BSIC', 'tech', 'lat', 'lon','azimuth','AvgNeighborDist', \
                                                            'MaxNeighborDist', 'NeighborsCount', 'Eng', 'elec_tilt','mech_tilt', 'site', \
                                                            'AdminCellState', 'Asset', 'Asset_Configuration', 'Cell_Type', 'Cell_Name_', \
                                                            'City', 'Height', 'market', 'Asset_Calc', 'Sector_uniq', 'FreqType', 'TAC', \
                                                            'RAC', 'band', 'vendor', 'CPICHPwr', 'MaxTransPwr', 'FreqMHz', 'hbw', \
                                                            'VBW', 'Antenna'])
cells_df = cells_df.drop_duplicates(subset='cell_cilac', keep="first")
cells_df = cells_df[['cell_name','cell_cilac', 'tech', 'lat', 'lon','azimuth', 'elec_tilt','mech_tilt', \
                      'site', 'market', 'band', 'vendor', 'hbw']]

# Changing 'tech' to 'sa', we need a siplified 'tech' field ['gsm', 'umts', 'lte', 'nsa', 'sa']
cells_df['tech'] = 'sa'
cells_df['tech_band'] = cells_df.apply(lambda x: str(x.tech) + "_" + str(x.band), axis = 1)

site_to_bands = (cells_df.groupby('site')['tech_band'].apply(lambda s: sorted(s.dropna().astype(str).unique().tolist())))

cells_df['site_tech_band_list'] = cells_df['site'].map(site_to_bands)

cells_df.head(2)

,cell_name,cell_cilac,tech,lat,lon,azimuth,elec_tilt,mech_tilt,site,market,band,vendor,hbw,tech_band,site_tech_band_list
0,KNTYS00368A_n70_AWS-4_UL5_1,41036398814,sa,35.720944,-84.022500,0,2.0,0,35.7209_-84.0225,KN,n70_AWS-4_UL5,NaN,61.5,sa_n70_AWS-4_UL5,"[sa_n29_E_DL, sa_n70_AWS-4_UL5, sa_n71_A, sa_n..."
1,KNTYS00052A_n70_AWS-4_UL5_2,41036402722,sa,35.754778,-83.932083,120,2.0,0,35.7548_-83.9321,KN,n70_AWS-4_UL5,NaN,61.5,sa_n70_AWS-4_UL5,"[sa_n29_E_DL, sa_n70_AWS-4_UL5, sa_n71_A, sa_n..."


#### Remove impacts to same cell

In [9]:
print("Number of impacts pre removal = {}".format(impacts_df.shape[0]))
impacts_df = impacts_df[impacts_df['cilac'] != impacts_df['next_cilac']].copy()
print("Number of impacts post removal = {}".format(impacts_df.shape[0]))

Number of impacts pre removal = 7806480
Number of impacts post removal = 7625621


#### Join impacts and cell GIS info

In [10]:
# Merge impacts_df and cells_df on impact cell
impacts_df = impacts_df.merge(cells_df, left_on='next_cilac', right_on='cell_cilac', how='left')
# Remove unrequired column
impacts_df.drop(columns = ['cell_cilac'], inplace = True)
# Rename columns
impacts_df.rename(columns = {'cell_name' : 'cell_impact_name', 'tech' : 'cell_impact_tech', 'lat' : 'cell_impact_lat', 'lon' : 'cell_impact_lon', 'azimuth' : 'cell_impact_azimuth', \
                             'elec_tilt' : 'cell_impact_elec_tilt',  'mech_tilt' : 'cell_impact_mech_tilt', 'site' : 'cell_impact_site', \
                             'band' : 'cell_impact_band', 'hbw' : 'cell_impact_hbw', 'market' : 'cell_impact_market', \
                             'vendor' : 'cell_impact_vendor', 'tech_band' : 'cell_impact_tech_band', \
                             'site_tech_band_list' : 'cell_impact_site_tech_band_list'}, inplace = True)

impacts_df = impacts_df.merge(cells_df, left_on='cilac', right_on='cell_cilac', how='left')
# Remove unrequired column
impacts_df.drop(columns = ['cell_cilac'], inplace = True)
# Rename columns
impacts_df.rename(columns = {'tech' : 'cell_tech', 'lat' : 'cell_lat', 'lon' : 'cell_lon', 'azimuth' : 'cell_azimuth', \
                             'elec_tilt' : 'cell_elec_tilt',  'mech_tilt' : 'cell_mech_tilt', 'site' : 'cell_site', \
                             'band' : 'cell_band', 'hbw' : 'cell_hbw', 'market' : 'cell_market', 'vendor' : 'cell_vendor', \
                              'tech_band' : 'cell_tech_band', 'site_tech_band_list' : 'cell_site_tech_band_list'}, inplace = True)
print("Number of impacts post join = {}".format(impacts_df.shape[0]))

Number of impacts post join = 7625621


In [11]:
impacts_df.head(2)

,tag,cilac,next_cilac,traffic_mobility,traffic_stationary,traffic_unknown,drop_mobility,drop_stationary,drop_unknown,voice_traffic_stationary,voice_traffic_mobility,voice_traffic_unknown,voice_drops_mobility,voice_drops_stationary,voice_drops_unknown,impact_time,drop_impact_time,cell_impact_name,cell_impact_tech,cell_impact_lat,cell_impact_lon,cell_impact_azimuth,cell_impact_elec_tilt,cell_impact_mech_tilt,cell_impact_site,cell_impact_market,cell_impact_band,cell_impact_vendor,cell_impact_hbw,cell_impact_tech_band,cell_impact_site_tech_band_list,cell_name,cell_tech,cell_lat,cell_lon,cell_azimuth,cell_elec_tilt,cell_mech_tilt,cell_site,cell_market,cell_band,cell_vendor,cell_hbw,cell_tech_band,cell_site_tech_band_list
0,-1,41396777666,41396777815,1,0,0,0,0,0,0,0,0,0,0,0,1,0,NJJER01817B_n66_G-H_1,sa,40.851994,-74.173914,20.0,3.5,0.0,40.852_-74.1739,NJ,n66_G-H,NaN,67.00112,sa_n66_G-H,"[sa_n66_G-H, sa_n70_AWS-4_UL5, sa_n71_F]",NJJER01587A_n70_AWS-4_UL15_2,sa,40.896079,-74.221470,190.0,2.0,0.0,40.8961_-74.2215,NJ,n70_AWS-4_UL5,NaN,66.98401,sa_n70_AWS-4_UL5,"[sa_n66_G-H, sa_n70_AWS-4_UL5, sa_n71_F]"
1,-1,41437720667,41437720920,35,28,4,0,0,0,1,0,1,0,0,0,567,0,NYNYC01421B_n71_E-F-G_3,sa,40.719253,-73.961133,210.0,2.0,2.0,40.7193_-73.9611,NY,n71_F,NaN,72.00000,sa_n71_F,"[sa_n66_G-H, sa_n70_AWS-4_UL5, sa_n71_F]",NYNYC01116A_n71_E-F-G_2,sa,40.729664,-73.951955,230.0,6.0,4.0,40.7297_-73.952,NY,n71_F,NaN,65.00000,sa_n71_F,"[sa_n66_G-H, sa_n70_AWS-4_UL5, sa_n71_F]"


#### If exist, remove N/A cells

In [12]:
print("Number of impacts pre removal of N/A cells = {}".format(impacts_df.shape[0]))
impacts_df = impacts_df[(~impacts_df["cell_name"].isna())].copy()
impacts_df = impacts_df[(~impacts_df["cell_impact_name"].isna())].copy()
print("Number of impacts post removal of N/A cells = {}".format(impacts_df.shape[0]))

Number of impacts pre removal of N/A cells = 7625621
Number of impacts post removal of N/A cells = 6672979


#### For this analysis keep Denver only

In [13]:
print("Number of impacts pre filter to Denver = {}".format(impacts_df.shape[0]))
impacts_df = impacts_df[(impacts_df.cell_market == "DN") & (impacts_df.cell_impact_market == "DN")].reset_index()
print("Number of impacts post filter to Denver = {}".format(impacts_df.shape[0]))

Number of impacts pre filter to Denver = 6672979
Number of impacts post filter to Denver = 231745


#### Check whether impact pair is co-sectored, co-sited

In [14]:
def azimuth_diff(a1, a2):
    diff = abs(a1 - a2) % 360
    return min(diff, 360 - diff)

impacts_df["co_site"] = impacts_df.apply(lambda x: "y" if x['cell_site'] == x['cell_impact_site'] else "n", axis = 1)

impacts_df["co_sectored"] = impacts_df.apply(lambda x: "y" if azimuth_diff(x["cell_azimuth"], x["cell_impact_azimuth"]) <= 20 else "n", axis = 1)


#### Sum traffic and drop columns

In [15]:
impacts_df["traffic_data"] = impacts_df.apply(lambda x: x['traffic_mobility'] + x['traffic_stationary'] + x['traffic_unknown'], axis = 1)

impacts_df["traffic_voice"] = impacts_df.apply(lambda x: x['voice_traffic_mobility'] + x['voice_traffic_stationary'] + \
                                                         x['voice_traffic_unknown'], axis = 1)

impacts_df["drops"] = impacts_df.apply(lambda x: x['drop_mobility'] + x['drop_stationary'] + x['drop_unknown'], axis = 1)

impacts_df["drops_voice"] = impacts_df.apply(lambda x: x['voice_drops_mobility'] + x['voice_drops_stationary'] + \
                                                       x['voice_drops_unknown'], axis = 1)

#### Create impacts per tech type - Note first we already changed fields 'cell_tech' and 'cell_impact_tech' to 'sa' to comply with formulas, need to think of how we standardize this..

In [16]:
# Create impact counters - Overall + per Tech
total_impacts_df = impacts_df[['cell_name', 'traffic_data', 'traffic_voice']].groupby(['cell_name']).sum().reset_index()
total_impacts_df.rename(columns = {'traffic_data':'total_cell_traffic_data', 'traffic_voice' : 'total_cell_traffic_voice'}, inplace = True)

gsm_exist  = True
umts_exist = True
lte_exist  = True
nsa_exist  = True
sa_exist   = True

# GSM
try:
    total_impacts_df_gsm = impacts_df[impacts_df['cell_impact_tech'] == "gsm"][['cell_name', 'traffic_data', 'traffic_voice']].groupby(['cell_name']).sum().reset_index()
    total_impacts_df_gsm.rename(columns = {'traffic_data':'total_cell_traffic_data_gsm', 'traffic_voice' : 'total_cell_traffic_voice_gsm'}, inplace = True)
except:
    gsm_exist = False
    
# UMTS
try:
    total_impacts_df_umts = impacts_df[impacts_df['cell_impact_tech'] == "umts"][['cell_name', 'traffic_data', 'traffic_voice']].groupby(['cell_name']).sum().reset_index()
    total_impacts_df_umts.rename(columns = {'traffic_data':'total_cell_traffic_data_umts', 'traffic_voice' : 'total_cell_traffic_voice_umts'}, inplace = True)
except:
    umts_exist = False

# LTE
try:
    total_impacts_df_lte = impacts_df[impacts_df['cell_impact_tech'] == "lte"][['cell_name', 'traffic_data', 'traffic_voice']].groupby(['cell_name']).sum().reset_index()
    total_impacts_df_lte.rename(columns = {'traffic_data':'total_cell_traffic_data_lte', 'traffic_voice' : 'total_cell_traffic_voice_lte'}, inplace = True)
except:
    lte_exist = False

# NSA
try:
    total_impacts_df_nsa = impacts_df[impacts_df['cell_impact_tech'] == "nsa"][['cell_name', 'traffic_data', 'traffic_voice']].groupby(['cell_name']).sum().reset_index()
    total_impacts_df_nsa.rename(columns = {'traffic_data':'total_cell_traffic_data_nsa', 'traffic_voice' : 'total_cell_traffic_voice_nsa'}, inplace = True)
except:
    nsa_exist = False

# SA
try:
    total_impacts_df_sa = impacts_df[impacts_df['cell_impact_tech'] == "sa"][['cell_name', 'traffic_data', 'traffic_voice']].groupby(['cell_name']).sum().reset_index()
    total_impacts_df_sa.rename(columns = {'traffic_data':'total_cell_traffic_data_sa', 'traffic_voice' : 'total_cell_traffic_voice_sa'}, inplace = True)
except:
    sa_exist = False

# Merge impact counter figures to 'impacts_df'
impacts_df = impacts_df.merge(total_impacts_df, left_on='cell_name', right_on='cell_name', how='left')
# Merge GSM counters
if gsm_exist  == True:
    impacts_df = impacts_df.merge(total_impacts_df_gsm, left_on='cell_name', right_on='cell_name', how='left')
else:
    impacts_df['total_cell_traffic_data_gsm'] = 0
    impacts_df['total_cell_traffic_voice_gsm'] = 0
# Merge UMTS counters
if umts_exist  == True:
    impacts_df = impacts_df.merge(total_impacts_df_umts, left_on='cell_name', right_on='cell_name', how='left')
else:
    impacts_df['total_cell_traffic_data_umts'] = 0
    impacts_df['total_cell_traffic_voice_umts'] = 0
# Merge LTE counters
if lte_exist  == True:
    impacts_df = impacts_df.merge(total_impacts_df_lte, left_on='cell_name', right_on='cell_name', how='left')
else:
    impacts_df['total_cell_traffic_data_lte'] = 0
    impacts_df['total_cell_traffic_voice_lte'] = 0
# Merge NSA counters
if nsa_exist  == True:
    impacts_df = impacts_df.merge(total_impacts_df_nsa, left_on='cell_name', right_on='cell_name', how='left')
else:
    impacts_df['total_cell_traffic_data_nsa'] = 0
    impacts_df['total_cell_traffic_voice_nsa'] = 0
# Merge SA counters
if sa_exist  == True:
    impacts_df = impacts_df.merge(total_impacts_df_sa, left_on='cell_name', right_on='cell_name', how='left')
else:
    impacts_df['total_cell_traffic_data_sa'] = 0
    impacts_df['total_cell_traffic_voice_sa'] = 0


#### Now find percentages per tech

In [17]:
def calcPercentage(num, denum):
	"""
	calculate percentage given num and denum
	"""
	try:
		if (int(num) != 0) & (int(denum) != 0):
			return round(100 * (num / denum), 2)
		else:
			return 0
	except:
		return 0

impacts_df["relation_impact_data(%)"] = impacts_df.apply(lambda x: calcPercentage(x['traffic_data'], x['total_cell_traffic_data']), axis = 1)
impacts_df["relation_impact_voice(%)"] = impacts_df.apply(lambda x: calcPercentage(x['traffic_voice'], x['total_cell_traffic_voice']), axis = 1)

impacts_df["total_gsm_impact_data(%)"] = impacts_df.apply(lambda x: calcPercentage(x['total_cell_traffic_data_gsm'], x['total_cell_traffic_data']), axis = 1)
impacts_df["total_umts_impact_data(%)"] = impacts_df.apply(lambda x: calcPercentage(x['total_cell_traffic_data_umts'], x['total_cell_traffic_data']), axis = 1)
impacts_df["total_lte_impact_data(%)"] = impacts_df.apply(lambda x: calcPercentage(x['total_cell_traffic_data_lte'], x['total_cell_traffic_data']), axis = 1)
impacts_df["total_nsa_impact_data(%)"] = impacts_df.apply(lambda x: calcPercentage(x['total_cell_traffic_data_nsa'], x['total_cell_traffic_data']), axis = 1)
impacts_df["total_sa_impact_data(%)"] = impacts_df.apply(lambda x: calcPercentage(x['total_cell_traffic_data_sa'], x['total_cell_traffic_data']), axis = 1)

impacts_df["total_gsm_impact_voice(%)"] = impacts_df.apply(lambda x: calcPercentage(x['total_cell_traffic_voice_gsm'], x['total_cell_traffic_voice']), axis = 1)
impacts_df["total_umts_impact_voice(%)"] = impacts_df.apply(lambda x: calcPercentage(x['total_cell_traffic_voice_umts'], x['total_cell_traffic_voice']), axis = 1)
impacts_df["total_lte_impact_voice(%)"] = impacts_df.apply(lambda x: calcPercentage(x['total_cell_traffic_voice_lte'], x['total_cell_traffic_voice']), axis = 1)
impacts_df["total_nsa_impact_voice(%)"] = impacts_df.apply(lambda x: calcPercentage(x['total_cell_traffic_voice_nsa'], x['total_cell_traffic_voice']), axis = 1)
impacts_df["total_sa_impact_voice(%)"] = impacts_df.apply(lambda x: calcPercentage(x['total_cell_traffic_voice_sa'], x['total_cell_traffic_voice']), axis = 1)


#### Repeate the above process over Tech-Band

In [18]:
for tech_band in impacts_df['cell_tech_band'].drop_duplicates().to_list():
    print("Working on tech-band = {}".format(tech_band))
    total_impacts_df_tech_band = impacts_df[impacts_df['cell_tech_band'] == tech_band][['cell_name', 'traffic_data', 'traffic_voice']].groupby(['cell_name']).sum().reset_index()
    total_impacts_df_tech_band.rename(columns = {'traffic_data':'total_cell_traffic_data_{}'.format(tech_band), 'traffic_voice' : 'total_cell_traffic_voice_{}'.format(tech_band)}, inplace = True)
    impacts_df = impacts_df.merge(total_impacts_df_tech_band, left_on='cell_name', right_on='cell_name', how='left')
    impacts_df["total_{}_impact_data(%)".format(tech_band)] = impacts_df.apply(lambda x: calcPercentage(x['total_cell_traffic_data_{}'.format(tech_band)], x['total_cell_traffic_data']), axis = 1)
    impacts_df["total_{}_impact_voice(%)".format(tech_band)] = impacts_df.apply(lambda x: calcPercentage(x['total_cell_traffic_voice_{}'.format(tech_band)], x['total_cell_traffic_voice']), axis = 1)


Working on tech-band = sa_n70_AWS-4_UL5
Working on tech-band = sa_n66_G
Working on tech-band = sa_n71_G
Working on tech-band = sa_n71_F-G
Working on tech-band = sa_n71_A
Working on tech-band = sa_n71_F


#### Get Distance columns

In [19]:
def getDist(lat1, lon1, lat2, lon2):
	"""
	Get the distance between 2 cells in meters 
	"""
	R = 6373.0 * 1000
	lat1 = math.radians(lat1)
	lon1 = math.radians(lon1)
	lat2 = math.radians(lat2)
	lon2 = math.radians(lon2)
	dlon = lon2 - lon1
	dlat = lat2 - lat1

	a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
	c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

	return R * c

# Get distance between inpact cells
impacts_df['distance'] = impacts_df.apply(lambda x: getDist(x.cell_lat, x.cell_lon, x.cell_impact_lat, x.cell_impact_lon), axis = 1)
		
# Get max distance between impact cell and furthest neighbour
max_distance_df = impacts_df[(impacts_df['relation_impact_data(%)'] >= 2) & (impacts_df['distance'] <= 30000)][['cell_name','distance']].groupby(['cell_name']).max().reset_index()
max_distance_df.rename(columns = {'distance':'max_neigh_distance'}, inplace = True)
impacts_df = impacts_df.merge(max_distance_df, on='cell_name', how='left')

# Get median distance between impact cell and it's neighbours
median_distance_df = impacts_df[(impacts_df['co_site'] == "n") & (impacts_df['relation_impact_data(%)'] >= 2) & (impacts_df['distance'] <= 30000)][['cell_name','distance']].groupby(['cell_name']).median().reset_index()
median_distance_df.rename(columns = {'distance':'median_neigh_distance'}, inplace = True)
impacts_df = impacts_df.merge(median_distance_df, on='cell_name', how='left')


#### Calculate angle between impact pair

In [16]:
def calculateAngle(lat1, long1, lat2, long2):
	"""
	Definition calculates angle between source and impact cell, returning angle to 'impacts_df' dataframe 

	"""
	dLon = (long2 - long1)
	
	y = math.sin(math.radians(dLon)) * math.cos(math.radians(lat2))
	x = math.cos(math.radians(lat1)) * math.sin(math.radians(lat2)) - math.sin(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.cos(math.radians(dLon))

	brng = np.arctan2(y, x)

	brng = np.degrees(brng)
	brng = (brng + 360) % 360
	#brng = 360 - brng # count degrees clockwise - remove to make counter-clockwise

	return brng

impacts_df['angle_cell_to_impact'] = impacts_df.apply(lambda x: calculateAngle(x.cell_lat, x.cell_lon, x.cell_impact_lat, x.cell_impact_lon), axis = 1)
impacts_df['angle_impact_to_cell'] = impacts_df.apply(lambda x: calculateAngle(x.cell_impact_lat, x.cell_impact_lon, x.cell_lat, x.cell_lon), axis = 1)

impacts_df['cell_azimuth_to_angle_diff'] = impacts_df.apply(lambda x: azimuth_diff(x.angle_cell_to_impact, x.cell_azimuth), axis = 1)
impacts_df['cell_impact_azimuth_to_angle_diff'] = impacts_df.apply(lambda x: azimuth_diff(x.angle_impact_to_cell , x.cell_impact_azimuth), axis = 1)


In [17]:
impacts_df.head()

,index,tag,cilac,next_cilac,traffic_mobility,traffic_stationary,traffic_unknown,drop_mobility,drop_stationary,drop_unknown,voice_traffic_stationary,voice_traffic_mobility,voice_traffic_unknown,voice_drops_mobility,voice_drops_stationary,voice_drops_unknown,impact_time,drop_impact_time,cell_impact_name,cell_impact_tech,cell_impact_lat,cell_impact_lon,cell_impact_azimuth,cell_impact_elec_tilt,cell_impact_mech_tilt,cell_impact_site,cell_impact_market,cell_impact_band,cell_impact_vendor,cell_impact_hbw,cell_impact_tech_band,cell_impact_site_tech_band_list,cell_name,cell_tech,cell_lat,cell_lon,cell_azimuth,cell_elec_tilt,cell_mech_tilt,cell_site,cell_market,cell_band,cell_vendor,cell_hbw,cell_tech_band,cell_site_tech_band_list,co_site,co_sectored,traffic_data,traffic_voice,drops,drops_voice,total_cell_traffic_data,total_cell_traffic_voice,total_cell_traffic_data_gsm,total_cell_traffic_voice_gsm,total_cell_traffic_data_umts,total_cell_traffic_voice_umts,total_cell_traffic_data_lte,total_cell_traffic_voice_lte,total_cell_traffic_data_nsa,total_cell_traffic_voice_nsa,total_cell_traffic_data_sa,total_cell_traffic_voice_sa,relation_impact_data(%),relation_impact_voice(%),total_gsm_impact_data(%),total_umts_impact_data(%),total_lte_impact_data(%),total_nsa_impact_data(%),total_sa_impact_data(%),total_gsm_impact_voice(%),total_umts_impact_voice(%),total_lte_impact_voice(%),total_nsa_impact_voice(%),total_sa_impact_voice(%),total_cell_traffic_data_sa_n70_AWS-4_UL5,total_cell_traffic_voice_sa_n70_AWS-4_UL5,total_sa_n70_AWS-4_UL5_impact_data(%),total_sa_n70_AWS-4_UL5_impact_voice(%),total_cell_traffic_data_sa_n66_G,total_cell_traffic_voice_sa_n66_G,total_sa_n66_G_impact_data(%),total_sa_n66_G_impact_voice(%),total_cell_traffic_data_sa_n71_G,total_cell_traffic_voice_sa_n71_G,total_sa_n71_G_impact_data(%),total_sa_n71_G_impact_voice(%),total_cell_traffic_data_sa_n71_F-G,total_cell_traffic_voice_sa_n71_F-G,total_sa_n71_F-G_impact_data(%),total_sa_n71_F-G_impact_voice(%),total_cell_traffic_data_sa_n71_A,total_cell_traffic_voice_sa_n71_A,total_sa_n71_A_impact_data(%),total_sa_n71_A_impact_voice(%),total_cell_traffic_data_sa_n71_F,total_cell_traffic_voice_sa_n71_F,total_sa_n71_F_impact_data(%),total_sa_n71_F_impact_voice(%),distance,max_neigh_distance,median_neigh_distance,angle_cell_to_impact,angle_impact_to_cell,cell_azimuth_to_angle_diff,cell_impact_azimuth_to_angle_diff
0,97,-1,4585818566,4585818771,1,0,0,0,0,0,0,0,0,0,0,0,1,0,DNDEN00051D_n71_F-G_3,sa,39.731486,-104.930944,190.0,4.0,0.0,39.7315_-104.931,DN,n71_F-G,NaN,62.00631,sa_n71_F-G,"[sa_n29_E_DL, sa_n66_G, sa_n70_AWS-4_UL5, sa_n...",DNDEN00141B_n70_AWS-4_UL15_2,sa,39.743755,-104.965646,120.0,2.0,0.0,39.7438_-104.966,DN,n70_AWS-4_UL5,NaN,61.500000,sa_n70_AWS-4_UL5,"[sa_n29_E_DL, sa_n66_G, sa_n70_AWS-4_UL5, sa_n...",n,n,1,0,0,0,10223,91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10223,91,0.01,0.0,0,0,0,0,100.0,0,0,0,0,100.0,10223.0,91.0,100.0,100.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,3266.873795,2443.300579,1227.495903,114.680410,294.702594,5.319590,104.702594
1,201,-1,4585818505,4585818708,0,1,0,0,0,0,0,0,0,0,0,0,11,0,DNDEN00163A_n71_F-G_3,sa,39.732209,-104.941152,240.0,3.0,0.0,39.7322_-104.941,DN,n71_F-G,NaN,63.00000,sa_n71_F-G,"[sa_n29_E_DL, sa_n66_G, sa_n70_AWS-4_UL5, sa_n...",DNDEN00124A_n66_G_1,sa,39.775778,-104.992833,0.0,2.0,0.0,39.7758_-104.993,DN,n66_G,NaN,66.000000,sa_n66_G,"[sa_n29_E_DL, sa_n66_G, sa_n70_AWS-4_UL5, sa_n...",n,n,1,0,0,0,13281,115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13281,115,0.01,0.0,0,0,0,0,100.0,0,0,0,0,100.0,NaN,NaN,0.0,0.0,13281.0,115.0,100.0,100.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,6558.700006,4261.400195,3982.785140,137.620608,317.653657,137.620608,77.653657
2,252,-1,4585855486,4585855494,2,103,0,0,0,0,0,0,0,0,0,0,1212,0,DNDEN00357A_n70_AWS-4_UL15_3,sa,38.880831,-104.821506,270.0,2.0,0.0,38.8808_-104.822,DN,n70_AWS-4_UL5,NaN,61.50000,sa_n70_AWS-4_UL5,"[sa_n29_E_DL, sa_n70_AWS-4_UL5, sa_n71_F, sa_n..

In [20]:
impacts_df.to_csv("{}denver-enriched-impacts.csv".format(output_path))